In [1]:
# IMPORTS AND COMMONLY USED FUNCTIONS
# ==> always run first, change server to match the desired one

import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# "global" | "asia" | "jpn" | "kor"
server = "global"


uid_df = pd.read_json('epic7_user_world_{}.json'.format(server))
users = dict()
for x in range(len(uid_df)):
    u = dict(uid_df.iloc[x]["users"])
    users[u["nick_no"]] = u["nick_nm"]

users2 = dict()
for x in range(len(uid_df)):
    u = dict(uid_df.iloc[x]["users"])
    users2[u["nick_nm"]] = u["nick_no"]
    
heroname_df = pd.read_json('heronames.json')
heronames = dict()
for x in range(len(heroname_df)):
    heronames[heroname_df.iloc[x]["code"]] = heroname_df.iloc[x]["name"]
    
def getHeroName(code):
    return heronames[code]
def getUID(name):
    return users2[name]
def getUsername(uid):
    return users[uid]

In [7]:
# LOOK FOR SUSPICIOUSLY SHORT GAMES
# max_turns: the amount of turns or less the game is allowed to last

def findShortGames(uid, max_turns=1):
    url = "https://epic7.gg.onstove.com/gameApi/getBattleList?nick_no={}&world_code=world_{}&lang=en&season_code=".format(uid, server)
    payload = {}
    response = requests.post(url, json=payload)

    filename = '{userId}.json'.format(userId=uid)
    if response.status_code == 200:
        json_data = response.json()
        with open(filename, 'w') as file:
            json.dump(json_data, file, indent=4)

        battles = pd.read_json(filename)
        battles = dict(battles.loc["battle_list"])['result_body']
        ffs = dict()
        for x in range(len(battles)):
            if battles[x]["season_code"] == "pvp_rta_ss11" and battles[x]["turn"] <= max_turns:
                if battles[x]["enemy_nick_no"] in ffs:
                    ffs[battles[x]["enemy_nick_no"]] = ffs[battles[x]["enemy_nick_no"]] + 1
                else: 
                    ffs[battles[x]["enemy_nick_no"]] = 1
        matches = dict()
        wins = dict()
        for x in range(len(battles)):
            for y in ffs.keys():
                if battles[x]["season_code"] == "pvp_rta_ss11" and battles[x]["enemy_nick_no"] == y:
                    if battles[x]["enemy_nick_no"] in matches:
                        matches[battles[x]["enemy_nick_no"]] = matches[battles[x]["enemy_nick_no"]] + 1
                    else: 
                        wins[battles[x]["enemy_nick_no"]] = 0
                        matches[battles[x]["enemy_nick_no"]] = 1

                    if battles[x]["iswin"] == 1:
                        wins[battles[x]["enemy_nick_no"]] = wins[battles[x]["enemy_nick_no"]] + 1
        print(users[str(uid)]+" - games ended before turn 2:")
        for y in ffs.keys():
            print("{}: {}/{} ({}), winrate against: {}".format(y, ffs[y], matches[y], str(100*ffs[y]/matches[y])+"%", str(100*wins[y]/matches[y])+"%"))
    else:
        print(f'Error: {response.status_code}')
        print(response.text)

In [9]:
# LOOK FOR SUSPICIOUS PICKS
# sus_log_p: the log probability limit for showing games
# -30 seems to be a decently good value


def generateData(uid, sus_log_p=-30):
    url = "https://epic7.gg.onstove.com/gameApi/getBattleList?nick_no={}&world_code=world_{}&lang=en&season_code=".format(uid, server)
    payload = {}
    response = requests.post(url, json=payload)


    filename = '{userId}.json'.format(userId=uid)
    if response.status_code == 200:
        json_data = response.json()
        with open(filename, 'w') as file:
            json.dump(json_data, file, indent=4)

        battles = pd.read_json(filename)
        battles = dict(battles.loc["battle_list"])['result_body']

        enemies = dict()
        heroes = dict()
        total_picks = 0
        total_battles = 0
        for x in range(len(battles)):
            if battles[x]["season_code"] == "pvp_rta_ss11":
                total_battles = total_battles + 1
                if battles[x]["enemy_nick_no"] in enemies:
                    enemies[battles[x]["enemy_nick_no"]] = enemies[battles[x]["enemy_nick_no"]] + 1
                else: 
                    enemies[battles[x]["enemy_nick_no"]] = 1    
                h = battles[x]["my_deck"]["hero_list"]
                for x2 in range(len(h)):
                    total_picks = total_picks + len(h)
                    if h[x2]["hero_code"] in heroes:
                        heroes[h[x2]["hero_code"]] = heroes[h[x2]["hero_code"]] + 1
                    else:
                        heroes[h[x2]["hero_code"]] = 1

        for hero in heroes.keys():
            heroes[hero] = np.log(heroes[hero]) - np.log(total_picks)

        battle_log_p = [1]*total_battles
        x5 = 0
        picks = [""]*total_battles
        for x in range(len(battles)):
            if battles[x]["season_code"] == "pvp_rta_ss11":   
                h = battles[x]["my_deck"]["hero_list"]
                for x2 in range(len(h)):
                    if x2 == 0:
                        picks[x5] = picks[x5] + getHeroName(h[x2]["hero_code"])
                    else:
                        picks[x5] = picks[x5] + ", " + getHeroName(h[x2]["hero_code"])
                    if h[x2]["hero_code"] in heroes:
                        battle_log_p[x5] = battle_log_p[x5] + heroes[h[x2]["hero_code"]]
                x5 = x5 + 1
        
        print("Generating data for: " + str(uid))
    else:
        print(f'Error: {response.status_code}')
        print(response.text)

In [2]:
rta_players = [159055138, 60090739, 63471324, 79680386, 60318328, 139771474,
               66879752, 63467428, 163085629, 188175150, 68183898, 60134130,
               85251058, 206705999, 124552294, 191603279, 177444551, 159620508,
               171276418, 170502668, 61214698, 133232375, 60142509, 85743517,
               191616064, 63908598, 70726715, 155256756, 134489859, 79806553]

In [12]:
for i in range(len(rta_players)):
    generateData(rta_players[i])

Generating data for: 159055138
Generating data for: 60090739
Generating data for: 63471324
Generating data for: 79680386
Generating data for: 60318328
Generating data for: 139771474
Generating data for: 66879752
Generating data for: 63467428
Generating data for: 163085629
Generating data for: 188175150
Generating data for: 68183898
Generating data for: 60134130
Generating data for: 85251058
Generating data for: 206705999
Generating data for: 124552294
Generating data for: 191603279
Generating data for: 177444551
Generating data for: 159620508
Generating data for: 171276418
Generating data for: 170502668
Generating data for: 61214698
Generating data for: 133232375
Generating data for: 60142509
Generating data for: 85743517
Generating data for: 191616064
Generating data for: 63908598
Generating data for: 70726715
Generating data for: 155256756
Generating data for: 134489859
Generating data for: 79806553


In [ ]:
# GET ALL GLOBAL PLAYER DATA

def getAllGlobalData():
    n = len(uid_df)
    uids2 = [0]*n
    n1 = 0
    for x in range(n):
        if int(dict(uid_df.iloc[x])["users"]["rank"]) == 70:
            n1 = n1 + 1
            uids2[x] = int(dict(uid_df.iloc[x])["users"]["nick_no"])

    uids = [0]*n1
    n2 = 0
    for x in range(n):
        if uids2[x] != 0:
            uids[n2] = uids2[x]
            n2 = n2 + 1

    print(len(uids))

    for x in range(n):
        uid = uids[x]
        url = "https://epic7.gg.onstove.com/gameApi/getBattleList?nick_no={userId}&world_code=world_global&lang=en&season_code="
        payload = {}
        response = requests.post(url.format(userId=uid), json=payload)

        if response.status_code == 200:
            # Parse the JSON response
            json_data = response.json()
            filename = '{userId}.json'.format(userId=uid)
            with open(filename, 'w') as file:
                json.dump(json_data, file, indent=4)
            print(x+"/"+n)
        else:
            print(f'Error: {response.status_code}')
            print(response.text)